##### Copyright 2019 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf
import sys

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
sys.argv = ["", input("Train Data filename:"), input("Test Data filename:")]

In [0]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

Load Data

In [0]:
!head {sys.argv[1]}

In [0]:
LABEL_COLUMN = 'chd'
LABELS = [0, 1]

Read CSV file and crate dataset

In [0]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(sys.argv[1])
raw_test_data = get_dataset(sys.argv[2])

In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [0]:
show_batch(raw_train_data)

In [0]:
CSV_COLUMNS = ['row.names', 'sbp', 'tobacco', 'ldl', 'adiposity', 'famhist', 'typea', 'obesity', 'alcohol', 'age', 'chd']

temp_dataset = get_dataset(sys.argv[1], column_names=CSV_COLUMNS)

show_batch(temp_dataset)

In [0]:
SELECT_COLUMNS = ['row.names', 'sbp', 'tobacco', 'ldl', 'adiposity', 'famhist', 'typea', 'obesity', 'alcohol', 'age', 'chd']

temp_dataset = get_dataset(sys.argv[1], select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

In [0]:
SELECT_COLUMNS = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age', 'chd']
DEFAULTS = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0]
temp_dataset = get_dataset(sys.argv[1], 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

In [0]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [0]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [0]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

In [0]:
show_batch(raw_train_data)

In [0]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [0]:
NUMERIC_FEATURES = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [0]:
show_batch(packed_train_data)

In [0]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [0]:
import pandas as pd
desc = pd.read_csv(sys.argv[1])[NUMERIC_FEATURES].describe()
desc

In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std


In [0]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

In [0]:
example_batch['numeric']

In [0]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

In [0]:
CATEGORIES = {
    'famhist': ['Absent', 'Present']
}


In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [0]:
# See what you just created.
categorical_columns

In [0]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [0]:
print(preprocessing_layer(example_batch).numpy()[0])

Build the model

In [0]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [0]:
from keras import regularizers
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, kernel_regularizer=regularizers.l2(0.0001), activation='elu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(128, kernel_regularizer=regularizers.l2(0.0001), activation='elu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(128, kernel_regularizer=regularizers.l2(0.0001), activation='elu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(128, kernel_regularizer=regularizers.l2(0.0001), activation='elu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

Train, evaluate, and predict

In [0]:
model.fit(train_data, epochs=100)
test_loss, test_accuracy = model.evaluate(test_data)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Infer labels

In [0]:
predictions = model.predict(test_data)

# Show some results
for prediction, chd in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted: {:.2%}".format(prediction[0]),
        " | chd: ",
        ("1" if bool(chd) else "0"))

